In [2]:
using Pkg
Pkg.instantiate()

    Updating registry at `/opt/julia/registries/General.toml`
   Installed RegistryInstances ─────── v0.1.0
   Installed Debugger ──────────────── v0.7.8
   Installed EpollShim_jll ─────────── v0.0.20230411+0
   Installed CodeTracking ──────────── v1.3.1
   Installed Highlights ────────────── v0.5.2
   Installed xkbcommon_jll ─────────── v1.4.1+1
   Installed GMT_jll ───────────────── v6.4.2+1
   Installed SpecialFunctions ──────── v2.3.1
   Installed libpng_jll ────────────── v1.6.38+1
   Installed ANSIColoredPrinters ───── v0.0.1
   Installed JuliaInterpreter ──────── v0.9.23
   Installed IOCapture ─────────────── v0.2.3
   Installed Wayland_jll ───────────── v1.21.0+1
   Installed MarkdownAST ───────────── v0.1.2
   Installed LazilyInitializedFields ─ v1.2.1
   Installed Documenter ────────────── v1.1.2
Precompiling project...
  ✓ IOCapture
  ✓ LazilyInitializedFields
  ✓ ANSIColoredPrinters
  ✓ EpollShim_jll
  ✓ libpng_jll
  ✓ CodeTracking
  ✓ MarkdownAST
  ✓ RegistryInstances
  ✓ 

In [3]:
using ArchGDAL
using FreqTables
using LinearAlgebra
using DataFrames
using Dates
using Pipe
using Plots
using Rasters
using Serialization
using SparseArrays

include("util.jl")
include("geo_ana.jl")
include("model_meta_pop.jl")
include("main_model_meta_pop.jl")

validate_meta_population_model

In [2]:
path_spatial = "../dt_tmp/spatial_params_agg230.ser"

"../dt_tmp/spatial_params_agg230.ser"

## International airport location, index 

In [5]:
sp_pars = deserialize(path_spatial)
df = sp_pars.df
# Tambo International
lat = -26.12825796201514
lon = 28.242074092511
dist = harversine_dist.(lat, lon, df[:, :lat], df[:, :lon]) 
println("argmin(dist): $(argmin(dist)), dist: $(dist[argmin(dist)])")

# Cape Town 
lat =  -33.970502228847884
lon = 18.600228711334545
dist = harversine_dist.(lat, lon, df[:, :lat], df[:, :lon]) 
println("argmin(dist): $(argmin(dist)), dist: $(dist[argmin(dist)])")

# King Shaka 
lat =  -29.608764960536764
lon = 31.115368797913593
dist = harversine_dist.(lat, lon, df[:, :lat], df[:, :lon]) 
println("argmin(dist): $(argmin(dist)), dist: $(dist[argmin(dist)])")



argmin(dist): 11, dist: 9.631110668981826
argmin(dist): 7, dist: 5.508420257007044
argmin(dist): 62, dist: 8.80101225388816


# Transmission model 

In [6]:
n_sim = 10_000

10000

In [7]:
R0 = 14.0
α = 0.05
pc = 0.25
pattern = "population_size"

"population_size"

In [8]:
imp_ws_moz = CSV.read("../data/imp_ws_moz.csv", DataFrame, header=false)[:,1]
imp_ws_airport = CSV.read("../data/imp_ws_airport.csv", DataFrame, header=false)[:,1]
nothing

In [9]:
include("model_meta_pop.jl")
include("main_model_meta_pop.jl")

validate_meta_population_model

In [10]:
function post_processing(path_trans, path_spatial, pc)
    par_AFP, par_ES = set_par_AFP_ES(path_spatial_params=path_spatial, pc=pc)
    path_save = sensitivity_all_summary(par_AFP, par_ES; path_trans=path_trans)
    #validate_meta_population_model(path_trans, path_spatial)
    remove_all_leaving_one(path_trans)
    return path_save
end

function run_all_patterns(;
        R0=1.0, α=0.05, pc=1.0, 
        imp_ws_moz=[1.0], imp_ws_airport=[1.0],
        n_sim=100,
        path_spatial=""
    )
    summary_info = []
    for pattern in ["population_size", "airport", "mozambique"]
        imp_ws = pattern=="airport" ? imp_ws_airport : imp_ws_moz
        path_trans = run_transmission_model(;
            R0=R0, α=α, pc=pc, imp_ws=imp_ws,
            n_sim=n_sim,
            path_spatial_params=path_spatial, 
            pattern=pattern
        )
        path_save = post_processing(path_trans, path_spatial, pc)
        rec = "R0=$(R0), α=$(α), pc=$(pc), n_sim=$(n_sim), pattern=$(pattern), path: $(path_save)"
        push!(summary_info, rec)
    end
    println(summary_info)
end

run_all_patterns (generic function with 1 method)

# Main results 

In [11]:
println("Main results==========================")
run_all_patterns(
    R0=R0, α=α, pc=pc, imp_ws_moz=imp_ws_moz, imp_ws_airport=imp_ws_airport, 
    n_sim=n_sim,
    path_spatial=path_spatial,
    )

Main results==========================
# of sites: 1517
SEIRMetaModelParams
  R0: Float64 14.0
  γ1: Float64 0.25
  γ2: Float64 0.06657789613848203
  σ: Float64 0.3291
  P_AFP: Float64 0.005
  N_tot: Array{Int64}((1517,)) [176661, 132438, 110952, 102279, 96377, 93257, 81002, 79789, 74368, 71796  …  103, 102, 102, 102, 102, 102, 102, 101, 101, 101]
  N_unvac: Array{Int64}((1517,)) [9009, 7004, 10801, 6729, 6341, 22933, 4284, 4219, 4893, 3975  …  4, 7, 7, 19, 19, 12, 12, 6, 7, 4]
  I0_init: Int64 1
  I0_ind: Int64 1
  days: Int64 1095
  n_site: Int64 1517
  α: Float64 0.05
  π_mat: Array{Float64}((1517, 1517)) [0.0 0.0008134913063202908 … 6.780709468953783e-7 4.898477234903362e-6; 0.0034150940977343493 0.0 … 4.753101976878339e-5 4.686095869451221e-6; … ; 1.9892426060623706e-6 0.00011196468975607586 … 0.0 3.3899886725864443e-9; 2.770651210640693e-5 4.626887381318493e-7 … 3.88258866975442e-10 0.0]
  μ: Float64 0.000547945205479452
  β: Float64 0.9320905459387484
  pc: Float64 0.25
  imp_ws

Progress: 100%|█████████████████████████████████████████| Time: 3:40:18m


13218.794741 seconds (114.96 M allocations: 32.793 TiB, 1.71% gc time, 0.00% compilation time)
Cum index:34
Actual ES coverage: 34.42009
AFPSurParams
  P_test: Float64 0.97
  P_sample: Float64 0.53
  P_H: Float64 0.9
ESParams
  g: Float64 0.23025850929940458
  P_test: Float64 0.97
  area: Array{Bool}((1517,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  n_freq: Int64 30


Progress: 100%|█████████████████████████████████████████| Time: 0:33:10


../dt_tmp_res/20231216_210437.ser
# of sites: 1517
SEIRMetaModelParams
  R0: Float64 14.0
  γ1: Float64 0.25
  γ2: Float64 0.06657789613848203
  σ: Float64 0.3291
  P_AFP: Float64 0.005
  N_tot: Array{Int64}((1517,)) [176661, 132438, 110952, 102279, 96377, 93257, 81002, 79789, 74368, 71796  …  103, 102, 102, 102, 102, 102, 102, 101, 101, 101]
  N_unvac: Array{Int64}((1517,)) [9009, 7004, 10801, 6729, 6341, 22933, 4284, 4219, 4893, 3975  …  4, 7, 7, 19, 19, 12, 12, 6, 7, 4]
  I0_init: Int64 1
  I0_ind: Int64 1
  days: Int64 1095
  n_site: Int64 1517
  α: Float64 0.05
  π_mat: Array{Float64}((1517, 1517)) [0.0 0.0008134913063202908 … 6.780709468953783e-7 4.898477234903362e-6; 0.0034150940977343493 0.0 … 4.753101976878339e-5 4.686095869451221e-6; … ; 1.9892426060623706e-6 0.00011196468975607586 … 0.0 3.3899886725864443e-9; 2.770651210640693e-5 4.626887381318493e-7 … 3.88258866975442e-10 0.0]
  μ: Float64 0.000547945205479452
  β: Float64 0.9320905459387484
  pc: Float64 0.25
  imp_ws: Arr

...

../dt_tmp_res/20231216_232227.ser
# of sites: 1517
SEIRMetaModelParams
  R0: Float64 14.0
  γ1: Float64 0.25
  γ2: Float64 0.06657789613848203
  σ: Float64 0.3291
  P_AFP: Float64 0.005
  N_tot: Array{Int64}((1517,)) [176661, 132438, 110952, 102279, 96377, 93257, 81002, 79789, 74368, 71796  …  103, 102, 102, 102, 102, 102, 102, 101, 101, 101]
  N_unvac: Array{Int64}((1517,)) [9009, 7004, 10801, 6729, 6341, 22933, 4284, 4219, 4893, 3975  …  4, 7, 7, 19, 19, 12, 12, 6, 7, 4]
  I0_init: Int64 1
  I0_ind: Int64 1
  days: Int64 1095
  n_site: Int64 1517
  α: Float64 0.05
  π_mat: Array{Float64}((1517, 1517)) [0.0 0.0008134913063202908 … 6.780709468953783e-7 4.898477234903362e-6; 0.0034150940977343493 0.0 … 4.753101976878339e-5 4.686095869451221e-6; … ; 1.9892426060623706e-6 0.00011196468975607586 … 0.0 3.3899886725864443e-9; 2.770651210640693e-5 4.626887381318493e-7 … 3.88258866975442e-10 0.0]
  μ: Float64 0.000547945205479452
  β: Float64 0.9320905459387484
  pc: Float64 0.25
  imp_ws: Arr

Progress: 100%|█████████████████████████████████████████| Time: 4:24:46


15886.381798 seconds (170.79 M allocations: 53.262 TiB, 0.56% gc time)
Cum index:34
Actual ES coverage: 34.42009
AFPSurParams
  P_test: Float64 0.97
  P_sample: Float64 0.53
  P_H: Float64 0.9
ESParams
  g: Float64 0.23025850929940458
  P_test: Float64 0.97
  area: Array{Bool}((1517,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  n_freq: Int64 30


Progress: 100%|█████████████████████████████████████████| Time: 0:31:17


../dt_tmp_res/20231217_041926.ser
Any["R0=14.0, α=0.05, pc=0.25, n_sim=10000, pattern=population_size, path: ../dt_tmp_res/20231216_210437.ser", "R0=14.0, α=0.05, pc=0.25, n_sim=10000, pattern=airport, path: ../dt_tmp_res/20231216_232227.ser", "R0=14.0, α=0.05, pc=0.25, n_sim=10000, pattern=mozambique, path: ../dt_tmp_res/20231217_041926.ser"]


## Sensitivity analysis 

In [12]:
println("Sensitivity analysis for R0 ==========================")
summary_info = []
pattern = "population_size"
for R0_sens in [10, 12, 16]
    path_trans = run_transmission_model(;
        R0=R0_sens, α=α, pc=pc, imp_ws=[1.0],
        n_sim=n_sim,
        path_spatial_params=path_spatial, 
        pattern=pattern
    )
    path_save = post_processing(path_trans, path_spatial, pc)
    rec = "R0=$(R0_sens), α=$(α), pc=$(pc), n_sim=$(n_sim), pattern=$(pattern), path: $(path_save)"
    push!(summary_info, rec)
end
println(summary_info)

Sensitivity analysis for R0 ==========================
# of sites: 1517
SEIRMetaModelParams
  R0: Float64 10.0
  γ1: Float64 0.25
  γ2: Float64 0.06657789613848203
  σ: Float64 0.3291
  P_AFP: Float64 0.005
  N_tot: Array{Int64}((1517,)) [176661, 132438, 110952, 102279, 96377, 93257, 81002, 79789, 74368, 71796  …  103, 102, 102, 102, 102, 102, 102, 101, 101, 101]
  N_unvac: Array{Int64}((1517,)) [9009, 7004, 10801, 6729, 6341, 22933, 4284, 4219, 4893, 3975  …  4, 7, 7, 19, 19, 12, 12, 6, 7, 4]
  I0_init: Int64 1
  I0_ind: Int64 1
  days: Int64 1095
  n_site: Int64 1517
  α: Float64 0.05
  π_mat: Array{Float64}((1517, 1517)) [0.0 0.0008134913063202908 … 6.780709468953783e-7 4.898477234903362e-6; 0.0034150940977343493 0.0 … 4.753101976878339e-5 4.686095869451221e-6; … ; 1.9892426060623706e-6 0.00011196468975607586 … 0.0 3.3899886725864443e-9; 2.770651210640693e-5 4.626887381318493e-7 … 3.88258866975442e-10 0.0]
  μ: Float64 0.000547945205479452
  β: Float64 0.6657789613848203
  pc: Float

Progress: 100%|█████████████████████████████████████████| Time: 1:33:18m


5598.260481 seconds (72.02 M allocations: 17.787 TiB, 0.47% gc time)
Cum index:34
Actual ES coverage: 34.42009
AFPSurParams
  P_test: Float64 0.97
  P_sample: Float64 0.53
  P_H: Float64 0.9
ESParams
  g: Float64 0.23025850929940458
  P_test: Float64 0.97
  area: Array{Bool}((1517,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  n_freq: Int64 30


Progress: 100%|█████████████████████████████████████████| Time: 0:31:37


../dt_tmp_res/20231217_062518.ser
# of sites: 1517
SEIRMetaModelParams
  R0: Float64 12.0
  γ1: Float64 0.25
  γ2: Float64 0.06657789613848203
  σ: Float64 0.3291
  P_AFP: Float64 0.005
  N_tot: Array{Int64}((1517,)) [176661, 132438, 110952, 102279, 96377, 93257, 81002, 79789, 74368, 71796  …  103, 102, 102, 102, 102, 102, 102, 101, 101, 101]
  N_unvac: Array{Int64}((1517,)) [9009, 7004, 10801, 6729, 6341, 22933, 4284, 4219, 4893, 3975  …  4, 7, 7, 19, 19, 12, 12, 6, 7, 4]
  I0_init: Int64 1
  I0_ind: Int64 1
  days: Int64 1095
  n_site: Int64 1517
  α: Float64 0.05
  π_mat: Array{Float64}((1517, 1517)) [0.0 0.0008134913063202908 … 6.780709468953783e-7 4.898477234903362e-6; 0.0034150940977343493 0.0 … 4.753101976878339e-5 4.686095869451221e-6; … ; 1.9892426060623706e-6 0.00011196468975607586 … 0.0 3.3899886725864443e-9; 2.770651210640693e-5 4.626887381318493e-7 … 3.88258866975442e-10 0.0]
  μ: Float64 0.000547945205479452
  β: Float64 0.7989347536617843
  pc: Float64 0.25
  imp_ws: Arr

...

../dt_tmp_res/20231217_105333.ser
# of sites: 1517
SEIRMetaModelParams
  R0: Float64 16.0
  γ1: Float64 0.25
  γ2: Float64 0.06657789613848203
  σ: Float64 0.3291
  P_AFP: Float64 0.005
  N_tot: Array{Int64}((1517,)) [176661, 132438, 110952, 102279, 96377, 93257, 81002, 79789, 74368, 71796  …  103, 102, 102, 102, 102, 102, 102, 101, 101, 101]
  N_unvac: Array{Int64}((1517,)) [9009, 7004, 10801, 6729, 6341, 22933, 4284, 4219, 4893, 3975  …  4, 7, 7, 19, 19, 12, 12, 6, 7, 4]
  I0_init: Int64 1
  I0_ind: Int64 1
  days: Int64 1095
  n_site: Int64 1517
  α: Float64 0.05
  π_mat: Array{Float64}((1517, 1517)) [0.0 0.0008134913063202908 … 6.780709468953783e-7 4.898477234903362e-6; 0.0034150940977343493 0.0 … 4.753101976878339e-5 4.686095869451221e-6; … ; 1.9892426060623706e-6 0.00011196468975607586 … 0.0 3.3899886725864443e-9; 2.770651210640693e-5 4.626887381318493e-7 … 3.88258866975442e-10 0.0]
  μ: Float64 0.000547945205479452
  β: Float64 1.0652463382157125
  pc: Float64 0.25
  imp_ws: Arr

Progress: 100%|█████████████████████████████████████████| Time: 5:39:08m40m


20348.158455 seconds (140.22 M allocations: 42.250 TiB, 0.49% gc time)
Cum index:34
Actual ES coverage: 34.42009
AFPSurParams
  P_test: Float64 0.97
  P_sample: Float64 0.53
  P_H: Float64 0.9
ESParams
  g: Float64 0.23025850929940458
  P_test: Float64 0.97
  area: Array{Bool}((1517,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  n_freq: Int64 30


Progress: 100%|█████████████████████████████████████████| Time: 0:32:01


../dt_tmp_res/20231217_171028.ser
Any["R0=10, α=0.05, pc=0.25, n_sim=10000, pattern=population_size, path: ../dt_tmp_res/20231217_062518.ser", "R0=12, α=0.05, pc=0.25, n_sim=10000, pattern=population_size, path: ../dt_tmp_res/20231217_105333.ser", "R0=16, α=0.05, pc=0.25, n_sim=10000, pattern=population_size, path: ../dt_tmp_res/20231217_171028.ser"]


In [13]:
println("Sensitivity analysis for α ==========================")
summary_info = []
for α_sens in [0.005, 0.010, 0.100, 0.500]
    path_trans = run_transmission_model(;
        R0=R0, α=α_sens, pc=pc, imp_ws=[1.0],
        n_sim=n_sim,
        path_spatial_params=path_spatial, 
        pattern=pattern,
    )
    path_save = post_processing(path_trans, path_spatial, pc)
    rec = "R0=$(R0), α=$(α_sens), pc=$(pc), n_sim=$(n_sim), pattern=$(pattern), path: $(path_save)"
    push!(summary_info, rec)
end
println(summary_info)

Sensitivity analysis for α ==========================
# of sites: 1517
SEIRMetaModelParams
  R0: Float64 14.0
  γ1: Float64 0.25
  γ2: Float64 0.06657789613848203
  σ: Float64 0.3291
  P_AFP: Float64 0.005
  N_tot: Array{Int64}((1517,)) [176661, 132438, 110952, 102279, 96377, 93257, 81002, 79789, 74368, 71796  …  103, 102, 102, 102, 102, 102, 102, 101, 101, 101]
  N_unvac: Array{Int64}((1517,)) [9009, 7004, 10801, 6729, 6341, 22933, 4284, 4219, 4893, 3975  …  4, 7, 7, 19, 19, 12, 12, 6, 7, 4]
  I0_init: Int64 1
  I0_ind: Int64 1
  days: Int64 1095
  n_site: Int64 1517
  α: Float64 0.005
  π_mat: Array{Float64}((1517, 1517)) [0.0 0.0008134913063202908 … 6.780709468953783e-7 4.898477234903362e-6; 0.0034150940977343493 0.0 … 4.753101976878339e-5 4.686095869451221e-6; … ; 1.9892426060623706e-6 0.00011196468975607586 … 0.0 3.3899886725864443e-9; 2.770651210640693e-5 4.626887381318493e-7 … 3.88258866975442e-10 0.0]
  μ: Float64 0.000547945205479452
  β: Float64 0.9320905459387484
  pc: Float

Progress: 100%|█████████████████████████████████████████| Time: 2:20:24m09


8424.623021 seconds (97.77 M allocations: 27.031 TiB, 0.34% gc time)
Cum index:34
Actual ES coverage: 34.42009
AFPSurParams
  P_test: Float64 0.97
  P_sample: Float64 0.53
  P_H: Float64 0.9
ESParams
  g: Float64 0.23025850929940458
  P_test: Float64 0.97
  area: Array{Bool}((1517,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  n_freq: Int64 30


Progress: 100%|█████████████████████████████████████████| Time: 0:33:25


../dt_tmp_res/20231217_200516.ser
# of sites: 1517
SEIRMetaModelParams
  R0: Float64 14.0
  γ1: Float64 0.25
  γ2: Float64 0.06657789613848203
  σ: Float64 0.3291
  P_AFP: Float64 0.005
  N_tot: Array{Int64}((1517,)) [176661, 132438, 110952, 102279, 96377, 93257, 81002, 79789, 74368, 71796  …  103, 102, 102, 102, 102, 102, 102, 101, 101, 101]
  N_unvac: Array{Int64}((1517,)) [9009, 7004, 10801, 6729, 6341, 22933, 4284, 4219, 4893, 3975  …  4, 7, 7, 19, 19, 12, 12, 6, 7, 4]
  I0_init: Int64 1
  I0_ind: Int64 1
  days: Int64 1095
  n_site: Int64 1517
  α: Float64 0.01
  π_mat: Array{Float64}((1517, 1517)) [0.0 0.0008134913063202908 … 6.780709468953783e-7 4.898477234903362e-6; 0.0034150940977343493 0.0 … 4.753101976878339e-5 4.686095869451221e-6; … ; 1.9892426060623706e-6 0.00011196468975607586 … 0.0 3.3899886725864443e-9; 2.770651210640693e-5 4.626887381318493e-7 … 3.88258866975442e-10 0.0]
  μ: Float64 0.000547945205479452
  β: Float64 0.9320905459387484
  pc: Float64 0.25
  imp_ws: Arr

...

../dt_tmp_res/20231218_031127.ser
# of sites: 1517
SEIRMetaModelParams
  R0: Float64 14.0
  γ1: Float64 0.25
  γ2: Float64 0.06657789613848203
  σ: Float64 0.3291
  P_AFP: Float64 0.005
  N_tot: Array{Int64}((1517,)) [176661, 132438, 110952, 102279, 96377, 93257, 81002, 79789, 74368, 71796  …  103, 102, 102, 102, 102, 102, 102, 101, 101, 101]
  N_unvac: Array{Int64}((1517,)) [9009, 7004, 10801, 6729, 6341, 22933, 4284, 4219, 4893, 3975  …  4, 7, 7, 19, 19, 12, 12, 6, 7, 4]
  I0_init: Int64 1
  I0_ind: Int64 1
  days: Int64 1095
  n_site: Int64 1517
  α: Float64 0.5
  π_mat: Array{Float64}((1517, 1517)) [0.0 0.0008134913063202908 … 6.780709468953783e-7 4.898477234903362e-6; 0.0034150940977343493 0.0 … 4.753101976878339e-5 4.686095869451221e-6; … ; 1.9892426060623706e-6 0.00011196468975607586 … 0.0 3.3899886725864443e-9; 2.770651210640693e-5 4.626887381318493e-7 … 3.88258866975442e-10 0.0]
  μ: Float64 0.000547945205479452
  β: Float64 0.9320905459387484
  pc: Float64 0.25
  imp_ws: Arra

Progress: 100%|█████████████████████████████████████████| Time: 3:05:05m


11105.075088 seconds (123.48 M allocations: 36.267 TiB, 0.28% gc time)
Cum index:34
Actual ES coverage: 34.42009
AFPSurParams
  P_test: Float64 0.97
  P_sample: Float64 0.53
  P_H: Float64 0.9
ESParams
  g: Float64 0.23025850929940458
  P_test: Float64 0.97
  area: Array{Bool}((1517,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  n_freq: Int64 30


Progress: 100%|█████████████████████████████████████████| Time: 0:33:05


../dt_tmp_res/20231218_065033.ser
Any["R0=14.0, α=0.005, pc=0.25, n_sim=10000, pattern=population_size, path: ../dt_tmp_res/20231217_200516.ser", "R0=14.0, α=0.01, pc=0.25, n_sim=10000, pattern=population_size, path: ../dt_tmp_res/20231217_233753.ser", "R0=14.0, α=0.1, pc=0.25, n_sim=10000, pattern=population_size, path: ../dt_tmp_res/20231218_031127.ser", "R0=14.0, α=0.5, pc=0.25, n_sim=10000, pattern=population_size, path: ../dt_tmp_res/20231218_065033.ser"]


In [14]:
println("Sensitivity analysis for pc ==========================")
summary_info = []
for pc_sens in [0.01, 0.05, 0.25, 0.50]
    path_trans = run_transmission_model(;
        R0=R0, α=α, pc=pc_sens, imp_ws=[1.0],
        n_sim=n_sim,
        path_spatial_params=path_spatial, 
        pattern=pattern,
    )
    path_save = post_processing(path_trans, path_spatial, pc_sens)
    rec = "R0=$(R0), α=$(α), pc=$(pc_sens), n_sim=$(n_sim), pattern=$(pattern), path: $(path_save)"
    push!(summary_info, rec)
end
println(summary_info)

Sensitivity analysis for pc ==========================
# of sites: 1517
SEIRMetaModelParams
  R0: Float64 14.0
  γ1: Float64 0.25
  γ2: Float64 0.06657789613848203
  σ: Float64 0.3291
  P_AFP: Float64 0.005
  N_tot: Array{Int64}((1517,)) [176661, 132438, 110952, 102279, 96377, 93257, 81002, 79789, 74368, 71796  …  103, 102, 102, 102, 102, 102, 102, 101, 101, 101]
  N_unvac: Array{Int64}((1517,)) [9009, 7004, 10801, 6729, 6341, 22933, 4284, 4219, 4893, 3975  …  4, 7, 7, 19, 19, 12, 12, 6, 7, 4]
  I0_init: Int64 1
  I0_ind: Int64 1
  days: Int64 1095
  n_site: Int64 1517
  α: Float64 0.05
  π_mat: Array{Float64}((1517, 1517)) [0.0 0.0008134913063202908 … 6.780709468953783e-7 4.898477234903362e-6; 0.0034150940977343493 0.0 … 4.753101976878339e-5 4.686095869451221e-6; … ; 1.9892426060623706e-6 0.00011196468975607586 … 0.0 3.3899886725864443e-9; 2.770651210640693e-5 4.626887381318493e-7 … 3.88258866975442e-10 0.0]
  μ: Float64 0.000547945205479452
  β: Float64 0.9320905459387484
  pc: Float

Progress: 100%|█████████████████████████████████████████| Time: 2:57:07m


10627.177128 seconds (116.46 M allocations: 33.738 TiB, 0.34% gc time)
Cum index:1517
Actual ES coverage: 100.0
AFPSurParams
  P_test: Float64 0.97
  P_sample: Float64 0.53
  P_H: Float64 0.9
ESParams
  g: Float64 0.23025850929940458
  P_test: Float64 0.97
  area: Array{Bool}((1517,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
  n_freq: Int64 30


Progress: 100%|█████████████████████████████████████████| Time: 0:41:34


../dt_tmp_res/20231218_103012.ser
# of sites: 1517
SEIRMetaModelParams
  R0: Float64 14.0
  γ1: Float64 0.25
  γ2: Float64 0.06657789613848203
  σ: Float64 0.3291
  P_AFP: Float64 0.005
  N_tot: Array{Int64}((1517,)) [176661, 132438, 110952, 102279, 96377, 93257, 81002, 79789, 74368, 71796  …  103, 102, 102, 102, 102, 102, 102, 101, 101, 101]
  N_unvac: Array{Int64}((1517,)) [9009, 7004, 10801, 6729, 6341, 22933, 4284, 4219, 4893, 3975  …  4, 7, 7, 19, 19, 12, 12, 6, 7, 4]
  I0_init: Int64 1
  I0_ind: Int64 1
  days: Int64 1095
  n_site: Int64 1517
  α: Float64 0.05
  π_mat: Array{Float64}((1517, 1517)) [0.0 0.0008134913063202908 … 6.780709468953783e-7 4.898477234903362e-6; 0.0034150940977343493 0.0 … 4.753101976878339e-5 4.686095869451221e-6; … ; 1.9892426060623706e-6 0.00011196468975607586 … 0.0 3.3899886725864443e-9; 2.770651210640693e-5 4.626887381318493e-7 … 3.88258866975442e-10 0.0]
  μ: Float64 0.000547945205479452
  β: Float64 0.9320905459387484
  pc: Float64 0.05
  imp_ws: Arr

...

../dt_tmp_res/20231218_180614.ser
# of sites: 1517
SEIRMetaModelParams
  R0: Float64 14.0
  γ1: Float64 0.25
  γ2: Float64 0.06657789613848203
  σ: Float64 0.3291
  P_AFP: Float64 0.005
  N_tot: Array{Int64}((1517,)) [176661, 132438, 110952, 102279, 96377, 93257, 81002, 79789, 74368, 71796  …  103, 102, 102, 102, 102, 102, 102, 101, 101, 101]
  N_unvac: Array{Int64}((1517,)) [9009, 7004, 10801, 6729, 6341, 22933, 4284, 4219, 4893, 3975  …  4, 7, 7, 19, 19, 12, 12, 6, 7, 4]
  I0_init: Int64 1
  I0_ind: Int64 1
  days: Int64 1095
  n_site: Int64 1517
  α: Float64 0.05
  π_mat: Array{Float64}((1517, 1517)) [0.0 0.0008134913063202908 … 6.780709468953783e-7 4.898477234903362e-6; 0.0034150940977343493 0.0 … 4.753101976878339e-5 4.686095869451221e-6; … ; 1.9892426060623706e-6 0.00011196468975607586 … 0.0 3.3899886725864443e-9; 2.770651210640693e-5 4.626887381318493e-7 … 3.88258866975442e-10 0.0]
  μ: Float64 0.000547945205479452
  β: Float64 0.9320905459387484
  pc: Float64 0.5
  imp_ws: Arra

Progress: 100%|█████████████████████████████████████████| Time: 3:41:56m


13316.588521 seconds (114.99 M allocations: 33.147 TiB, 0.30% gc time)
Cum index:9
Actual ES coverage: 16.731243
AFPSurParams
  P_test: Float64 0.97
  P_sample: Float64 0.53
  P_H: Float64 0.9
ESParams
  g: Float64 0.23025850929940458
  P_test: Float64 0.97
  area: Array{Bool}((1517,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  n_freq: Int64 30


Progress: 100%|█████████████████████████████████████████| Time: 0:52:00


../dt_tmp_res/20231218_224110.ser
Any["R0=14.0, α=0.05, pc=0.01, n_sim=10000, pattern=population_size, path: ../dt_tmp_res/20231218_103012.ser", "R0=14.0, α=0.05, pc=0.05, n_sim=10000, pattern=population_size, path: ../dt_tmp_res/20231218_140721.ser", "R0=14.0, α=0.05, pc=0.25, n_sim=10000, pattern=population_size, path: ../dt_tmp_res/20231218_180614.ser", "R0=14.0, α=0.05, pc=0.5, n_sim=10000, pattern=population_size, path: ../dt_tmp_res/20231218_224110.ser"]


## Mozambique risk ordered ES. 

In [15]:
imp_ws_moz_sorted = CSV.read("../data/imp_ws_moz_sorted.csv", DataFrame, header=false)[:,1]
imp_ws_airport_sorted = CSV.read("../data/imp_ws_airport_sorted.csv", DataFrame, header=false)[:,1]
path_spatial_sorted = "../dt_tmp/spatial_params_agg230_moz_sorted.ser"


"../dt_tmp/spatial_params_agg230_moz_sorted.ser"

In [16]:
println("Sensitivity analysis for Mozambique risk ordered ES ==========================")
run_all_patterns(
    R0=R0, α=α, pc=pc, 
    imp_ws_moz=imp_ws_moz_sorted, 
    imp_ws_airport=imp_ws_airport_sorted,
    n_sim=n_sim,
    path_spatial=path_spatial_sorted,
    )

Sensitivity analysis for Mozambique risk ordered ES ==========================
# of sites: 1517
SEIRMetaModelParams
  R0: Float64 14.0
  γ1: Float64 0.25
  γ2: Float64 0.06657789613848203
  σ: Float64 0.3291
  P_AFP: Float64 0.005
  N_tot: Array{Int64}((1517,)) [16942, 6206, 11638, 17796, 9978, 18076, 14578, 18809, 13780, 6951  …  115, 107, 107, 110, 101, 105, 102, 105, 101, 103]
  N_unvac: Array{Int64}((1517,)) [2049, 750, 1407, 2152, 1206, 2186, 1763, 2274, 1666, 840  …  7, 2, 6, 6, 7, 6, 7, 6, 6, 6]
  I0_init: Int64 1
  I0_ind: Int64 1
  days: Int64 1095
  n_site: Int64 1517
  α: Float64 0.05
  π_mat: Array{Float64}((1517, 1517)) [0.0 0.09700668215283446 … 6.40008051061156e-8 6.351348570063297e-8; 0.40108404561682365 0.0 … 2.3434779415237e-8 2.3265534898956926e-8; … ; 5.945035894425645e-8 2.2301115523520442e-8 … 0.0 3.73407299880895e-8; 6.393101238222124e-8 2.4169055529270278e-8 … 3.661928733425369e-8 0.0]
  μ: Float64 0.000547945205479452
  β: Float64 0.9320905459387484
  pc: Float

Progress: 100%|█████████████████████████████████████████| Time: 2:54:28m11m


10469.794992 seconds (113.71 M allocations: 32.749 TiB, 0.33% gc time)
Cum index:107
Actual ES coverage: 34.291977907735735
AFPSurParams
  P_test: Float64 0.97
  P_sample: Float64 0.53
  P_H: Float64 0.9
ESParams
  g: Float64 0.23025850929940458
  P_test: Float64 0.97
  area: Array{Bool}((1517,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  n_freq: Int64 30


Progress: 100%|█████████████████████████████████████████| Time: 0:32:07


../dt_tmp_res/20231219_021018.ser
# of sites: 1517
SEIRMetaModelParams
  R0: Float64 14.0
  γ1: Float64 0.25
  γ2: Float64 0.06657789613848203
  σ: Float64 0.3291
  P_AFP: Float64 0.005
  N_tot: Array{Int64}((1517,)) [16942, 6206, 11638, 17796, 9978, 18076, 14578, 18809, 13780, 6951  …  115, 107, 107, 110, 101, 105, 102, 105, 101, 103]
  N_unvac: Array{Int64}((1517,)) [2049, 750, 1407, 2152, 1206, 2186, 1763, 2274, 1666, 840  …  7, 2, 6, 6, 7, 6, 7, 6, 6, 6]
  I0_init: Int64 1
  I0_ind: Int64 1
  days: Int64 1095
  n_site: Int64 1517
  α: Float64 0.05
  π_mat: Array{Float64}((1517, 1517)) [0.0 0.09700668215283446 … 6.40008051061156e-8 6.351348570063297e-8; 0.40108404561682365 0.0 … 2.3434779415237e-8 2.3265534898956926e-8; … ; 5.945035894425645e-8 2.2301115523520442e-8 … 0.0 3.73407299880895e-8; 6.393101238222124e-8 2.4169055529270278e-8 … 3.661928733425369e-8 0.0]
  μ: Float64 0.000547945205479452
  β: Float64 0.9320905459387484
  pc: Float64 0.25
  imp_ws: Array{Float64}((1517,)) [0.

...

../dt_tmp_res/20231219_041910.ser
# of sites: 1517
SEIRMetaModelParams
  R0: Float64 14.0
  γ1: Float64 0.25
  γ2: Float64 0.06657789613848203
  σ: Float64 0.3291
  P_AFP: Float64 0.005
  N_tot: Array{Int64}((1517,)) [16942, 6206, 11638, 17796, 9978, 18076, 14578, 18809, 13780, 6951  …  115, 107, 107, 110, 101, 105, 102, 105, 101, 103]
  N_unvac: Array{Int64}((1517,)) [2049, 750, 1407, 2152, 1206, 2186, 1763, 2274, 1666, 840  …  7, 2, 6, 6, 7, 6, 7, 6, 6, 6]
  I0_init: Int64 1
  I0_ind: Int64 1
  days: Int64 1095
  n_site: Int64 1517
  α: Float64 0.05
  π_mat: Array{Float64}((1517, 1517)) [0.0 0.09700668215283446 … 6.40008051061156e-8 6.351348570063297e-8; 0.40108404561682365 0.0 … 2.3434779415237e-8 2.3265534898956926e-8; … ; 5.945035894425645e-8 2.2301115523520442e-8 … 0.0 3.73407299880895e-8; 6.393101238222124e-8 2.4169055529270278e-8 … 3.661928733425369e-8 0.0]
  μ: Float64 0.000547945205479452
  β: Float64 0.9320905459387484
  pc: Float64 0.25
  imp_ws: Array{Float64}((1517,)) [0.

Progress: 100%|█████████████████████████████████████████| Time: 4:38:43m


16723.175386 seconds (167.74 M allocations: 52.143 TiB, 0.21% gc time)
Cum index:107
Actual ES coverage: 34.291977907735735
AFPSurParams
  P_test: Float64 0.97
  P_sample: Float64 0.53
  P_H: Float64 0.9
ESParams
  g: Float64 0.23025850929940458
  P_test: Float64 0.97
  area: Array{Bool}((1517,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  n_freq: Int64 30


Progress: 100%|█████████████████████████████████████████| Time: 0:36:05


../dt_tmp_res/20231219_093454.ser
Any["R0=14.0, α=0.05, pc=0.25, n_sim=10000, pattern=population_size, path: ../dt_tmp_res/20231219_021018.ser", "R0=14.0, α=0.05, pc=0.25, n_sim=10000, pattern=airport, path: ../dt_tmp_res/20231219_041910.ser", "R0=14.0, α=0.05, pc=0.25, n_sim=10000, pattern=mozambique, path: ../dt_tmp_res/20231219_093454.ser"]


# Detection sensitivity, sensitivity analysis

In [ ]:
println("Sensitivity analysis for pop90 ==========================")
summary_info = []
path_trans = run_transmission_model(;
    R0=R0, α=α, pc=pc, imp_ws=[1.0],
    n_sim=n_sim,
    path_spatial_params=path_spatial, 
    pattern=pattern
)
for pop90 in [1, 30, 100, 300]
    g = - log(1-0.9)/pop90
    par_AFP, par_ES = set_par_AFP_ES(path_spatial_params=path_spatial, pc=pc, g=g)

    path_save = sensitivity_all_summary(par_AFP, par_ES; path_trans=path_trans)
    rec = "R0=$(R0), α=$(α), pc=$(pc), n_sim=$(n_sim), pattern=$(pattern), pop90=$(pop90), path: $(path_save)"
    push!(summary_info, rec)
end
remove_all_leaving_one(path_trans)
print(summary_info)

Sensitivity analysis for pop90 ==========================
# of sites: 1517
SEIRMetaModelParams
  R0: Float64 14.0
  γ1: Float64 0.25
  γ2: Float64 0.06657789613848203
  σ: Float64 0.3291
  P_AFP: Float64 0.005
  N_tot: Array{Int64}((1517,)) [176661, 132438, 110952, 102279, 96377, 93257, 81002, 79789, 74368, 71796  …  103, 102, 102, 102, 102, 102, 102, 101, 101, 101]
  N_unvac: Array{Int64}((1517,)) [9009, 7004, 10801, 6729, 6341, 22933, 4284, 4219, 4893, 3975  …  4, 7, 7, 19, 19, 12, 12, 6, 7, 4]
  I0_init: Int64 1
  I0_ind: Int64 1
  days: Int64 1095
  n_site: Int64 1517
  α: Float64 0.05
  π_mat: Array{Float64}((1517, 1517)) [0.0 0.0008134913063202908 … 6.780709468953783e-7 4.898477234903362e-6; 0.0034150940977343493 0.0 … 4.753101976878339e-5 4.686095869451221e-6; … ; 1.9892426060623706e-6 0.00011196468975607586 … 0.0 3.3899886725864443e-9; 2.770651210640693e-5 4.626887381318493e-7 … 3.88258866975442e-10 0.0]
  μ: Float64 0.000547945205479452
  β: Float64 0.9320905459387484
  pc: Fl

Progress: 100%|█████████████████████████████████████████| Time: 3:27:37m


12411.368750 seconds (113.94 M allocations: 32.793 TiB, 0.42% gc time)
Cum index:34
Actual ES coverage: 34.42009
AFPSurParams
  P_test: Float64 0.97
  P_sample: Float64 0.53
  P_H: Float64 0.9
ESParams
  g: Float64 2.302585092994046
  P_test: Float64 0.97
  area: Array{Bool}((1517,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  n_freq: Int64 30


Progress: 100%|█████████████████████████████████████████| Time: 0:48:51


../dt_tmp_res/20231219_135242.ser
Cum index:34
Actual ES coverage: 34.42009
AFPSurParams
  P_test: Float64 0.97
  P_sample: Float64 0.53
  P_H: Float64 0.9
ESParams
  g: Float64 0.07675283643313487
  P_test: Float64 0.97
  area: Array{Bool}((1517,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  n_freq: Int64 30


...

../dt_tmp_res/20231219_143417.ser
Cum index:34
Actual ES coverage: 34.42009
AFPSurParams
  P_test: Float64 0.97
  P_sample: Float64 0.53
  P_H: Float64 0.9
ESParams
  g: Float64 0.02302585092994046
  P_test: Float64 0.97
  area: Array{Bool}((1517,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  n_freq: Int64 30


Progress: 100%|█████████████████████████████████████████| Time: 0:40:51


../dt_tmp_res/20231219_151509.ser
Cum index:34
Actual ES coverage: 34.42009
AFPSurParams
  P_test: Float64 0.97
  P_sample: Float64 0.53
  P_H: Float64 0.9
ESParams
  g: Float64 0.0076752836433134864
  P_test: Float64 0.97
  area: Array{Bool}((1517,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  n_freq: Int64 30


Progress: 100%|█████████████████████████████████████████| Time: 0:37:38


../dt_tmp_res/20231219_155248.ser
Any["R0=14.0, α=0.05, pc=0.25, n_sim=10000, pattern=population_size, pop90=1, path: ../dt_tmp_res/20231219_135242.ser", "R0=14.0, α=0.05, pc=0.25, n_sim=10000, pattern=population_size, pop90=30, path: ../dt_tmp_res/20231219_143417.ser", "R0=14.0, α=0.05, pc=0.25, n_sim=10000, pattern=population_size, pop90=100, path: ../dt_tmp_res/20231219_151509.ser", "R0=14.0, α=0.05, pc=0.25, n_sim=10000, pattern=population_size, pop90=300, path: ../dt_tmp_res/20231219_155248.ser"]

# Sampling frequency sensitivity

In [ ]:
println("Sensitivity analysis for sampling frequency ==========================")
summary_info = []
path_trans = run_transmission_model(;
    R0=R0, α=α, pc=pc, imp_ws=[1.0],
    n_sim=n_sim,
    path_spatial_params=path_spatial, 
    pattern=pattern
)
for n_freq in [1, 7, 14, 60]
    par_AFP, par_ES = set_par_AFP_ES(path_spatial_params=path_spatial, pc=pc, n_freq=n_freq)
    path_save = sensitivity_all_summary(par_AFP, par_ES; path_trans=path_trans)
    rec = "R0=$(R0), α=$(α), pc=$(pc), n_sim=$(n_sim), pattern=$(pattern), n_freq=$(n_freq), path: $(path_save)"
    push!(summary_info, rec)
end
remove_all_leaving_one(path_trans)
print(summary_info)

Sensitivity analysis for sampling frequency ==========================
# of sites: 1517
SEIRMetaModelParams
  R0: Float64 14.0
  γ1: Float64 0.25
  γ2: Float64 0.06657789613848203
  σ: Float64 0.3291
  P_AFP: Float64 0.005
  N_tot: Array{Int64}((1517,)) [176661, 132438, 110952, 102279, 96377, 93257, 81002, 79789, 74368, 71796  …  103, 102, 102, 102, 102, 102, 102, 101, 101, 101]
  N_unvac: Array{Int64}((1517,)) [9009, 7004, 10801, 6729, 6341, 22933, 4284, 4219, 4893, 3975  …  4, 7, 7, 19, 19, 12, 12, 6, 7, 4]
  I0_init: Int64 1
  I0_ind: Int64 1
  days: Int64 1095
  n_site: Int64 1517
  α: Float64 0.05
  π_mat: Array{Float64}((1517, 1517)) [0.0 0.0008134913063202908 … 6.780709468953783e-7 4.898477234903362e-6; 0.0034150940977343493 0.0 … 4.753101976878339e-5 4.686095869451221e-6; … ; 1.9892426060623706e-6 0.00011196468975607586 … 0.0 3.3899886725864443e-9; 2.770651210640693e-5 4.626887381318493e-7 … 3.88258866975442e-10 0.0]
  μ: Float64 0.000547945205479452
  β: Float64 0.932090545938

Progress: 100%|█████████████████████████████████████████| Time: 3:08:29m


11309.776701 seconds (113.89 M allocations: 32.793 TiB, 0.43% gc time)
Cum index:34
Actual ES coverage: 34.42009
AFPSurParams
  P_test: Float64 0.97
  P_sample: Float64 0.53
  P_H: Float64 0.9
ESParams
  g: Float64 0.23025850929940458
  P_test: Float64 0.97
  area: Array{Bool}((1517,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  n_freq: Int64 1


Progress: 100%|█████████████████████████████████████████| Time: 2:26:59m


../dt_tmp_res/20231219_212930.ser
Cum index:34
Actual ES coverage: 34.42009
AFPSurParams
  P_test: Float64 0.97
  P_sample: Float64 0.53
  P_H: Float64 0.9
ESParams
  g: Float64 0.23025850929940458
  P_test: Float64 0.97
  area: Array{Bool}((1517,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  n_freq: Int64 7


...

../dt_tmp_res/20231219_221654.ser
Cum index:34
Actual ES coverage: 34.42009
AFPSurParams
  P_test: Float64 0.97
  P_sample: Float64 0.53
  P_H: Float64 0.9
ESParams
  g: Float64 0.23025850929940458
  P_test: Float64 0.97
  area: Array{Bool}((1517,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  n_freq: Int64 14


Progress: 100%|█████████████████████████████████████████| Time: 0:38:43


../dt_tmp_res/20231219_225537.ser
Cum index:34
Actual ES coverage: 34.42009
AFPSurParams
  P_test: Float64 0.97
  P_sample: Float64 0.53
  P_H: Float64 0.9
ESParams
  g: Float64 0.23025850929940458
  P_test: Float64 0.97
  area: Array{Bool}((1517,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  n_freq: Int64 60


Progress: 100%|█████████████████████████████████████████| Time: 0:36:08


../dt_tmp_res/20231219_233146.ser
Any["R0=14.0, α=0.05, pc=0.25, n_sim=10000, pattern=population_size, n_freq=1, path: ../dt_tmp_res/20231219_212930.ser", "R0=14.0, α=0.05, pc=0.25, n_sim=10000, pattern=population_size, n_freq=7, path: ../dt_tmp_res/20231219_221654.ser", "R0=14.0, α=0.05, pc=0.25, n_sim=10000, pattern=population_size, n_freq=14, path: ../dt_tmp_res/20231219_225537.ser", "R0=14.0, α=0.05, pc=0.25, n_sim=10000, pattern=population_size, n_freq=60, path: ../dt_tmp_res/20231219_233146.ser"]